In [12]:
import os
import numpy as np
import cv2
import pandas as pd
from sklearn.metrics import confusion_matrix

In [13]:
def compute_f1(df):
    TP = df.loc["real_landslide", "pred_landslide"]
    FP = df.loc["real_background", "pred_landslide"]
    TN = df.loc["real_background", "pred_background"]
    FN = df.loc["real_landslide", "pred_background"]

    try:
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    except Exception:
        f1_score = 0

    return f1_score, (TP, FP, TN, FN)

## 1. Valutazione Patch-Wise
In questa valutazione fornita dal professore, se un patch (immagine intera) ha un numero di pixel frana superiore al `THRESHOLD` (1), l'intero patch è considerato positivo (1). Altrimenti negativo (0).

In [21]:
results = []
THRESHOLD = 1
percentage = False

PATH_REAL_MASK = f"original"
PATH_PRED_MASK = f"predicted"
zone = "A19"

real_files = sorted(os.listdir(PATH_REAL_MASK))
pred_files = sorted(os.listdir(PATH_PRED_MASK))

y_true = []
y_pred = []

for real_file, pred_file in zip(real_files, pred_files):
    real_path = os.path.join(PATH_REAL_MASK, real_file)
    pred_path = os.path.join(PATH_PRED_MASK, pred_file)

    real_mask = cv2.imread(real_path, cv2.IMREAD_GRAYSCALE)
    pred_mask = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)

    real_count = np.sum(real_mask == 255)
    pred_count = np.sum(pred_mask == 255)

    if percentage:
        total_pixels = real_mask.size
        real_frac = real_count / total_pixels
        pred_frac = pred_count / total_pixels

        y_true.append(int(real_frac >= THRESHOLD))
        y_pred.append(int(pred_frac >= THRESHOLD))
    else:
        y_true.append(int(real_count >= THRESHOLD))
        y_pred.append(int(pred_count >= THRESHOLD))

cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
cm = pd.DataFrame(cm, index=["real_background", "real_landslide"], columns=["pred_background", "pred_landslide"])

f1_score, (TP, FP, TN, FN) = compute_f1(cm)
precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
results.append((zone, TP, FP, TN, FN, THRESHOLD, percentage, precision, recall, f1_score))
res = pd.DataFrame(results, columns=["zone","TP", "FP", "TN", "FN", "threshold", "percentage", "precision", "recall", "f1(landslide)"])
res

,zone,TP,FP,TN,FN,threshold,percentage,precision,recall,f1(landslide)
0,A19,6,0,10,0,1,False,1.0,1.0,1.0


---
## 2. Valutazione Pixel-Wise
In questa valutazione, calcoliamo True Positives, False Positives ecc. sui **singoli pixel** di ogni immagine. Un pixel frana = 1, pixel sfondo = 0. Questo è il modo classico per valutare la segmentazione.

In [ ]:
# === AGGIUNTA: Valutazione PIXEL-WISE ===
# Questo calcola i True Positives ecc. sui singoli pixel frana vs sfondo (valori 0 e 255)
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

results_pixel = []
PATH_REAL_MASK = "original"
PATH_PRED_MASK = "predicted"
zone = "A19"

real_files = sorted(os.listdir(PATH_REAL_MASK))
pred_files = sorted(os.listdir(PATH_PRED_MASK))

y_true_pixel = []
y_pred_pixel = []

for real_file, pred_file in zip(real_files, pred_files):
    real_path = os.path.join(PATH_REAL_MASK, real_file)
    pred_path = os.path.join(PATH_PRED_MASK, pred_file)

    # Leggi come in Scala di Grigi
    real_mask = cv2.imread(real_path, cv2.IMREAD_GRAYSCALE)
    pred_mask = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
    
    # Standardizza a (0 e 1)
    real_mask_bin = (real_mask > 127).astype(np.uint8).flatten()
    pred_mask_bin = (pred_mask > 127).astype(np.uint8).flatten()
    
    y_true_pixel.extend(real_mask_bin)
    y_pred_pixel.extend(pred_mask_bin)

# Calcola CM
cm_pixel = confusion_matrix(y_true_pixel, y_pred_pixel, labels=[0, 1])
cm_pixel_df = pd.DataFrame(cm_pixel, index=["real_background", "real_landslide"], columns=["pred_background", "pred_landslide"])

def compute_f1(df):
    TP = df.loc["real_landslide", "pred_landslide"]
    FP = df.loc["real_background", "pred_landslide"]
    TN = df.loc["real_background", "pred_background"]
    FN = df.loc["real_landslide", "pred_background"]
    try:
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    except Exception:
        f1_score = 0
    return f1_score, (TP, FP, TN, FN)

f1_score, (TP, FP, TN, FN) = compute_f1(cm_pixel_df)
precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0

results_pixel.append((zone, TP, FP, TN, FN, precision, recall, f1_score))
res_pixel = pd.DataFrame(results_pixel, columns=["zone","TP", "FP", "TN", "FN", "precision", "recall", "f1(landslide)"])
print("--- RISULTATI PIXEL-WISE ---")
display(res_pixel)
